In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain_community.document_loaders.pdf import UnstructuredPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
loader = UnstructuredPDFLoader("doc/sample.pdf")
raw_docs = loader.load()
raw_docs

[Document(page_content='投資信託説明書 （交付目論見書）\n\n使用開始日 2024.１.19\n\n自分だけの投資スタイル、探せる、見つかる。\n\nノーロード・インデックスファンド・シリーズ。\n\nｅＭＡＸＩＳ専用サイト\u3000 https://emaxis.jp/\n\nファンドのiDeCo取扱先は「eMAXIS SlimシリーズiDeCo取扱先」まで https://www.am.mufg.jp/service/ideco/sales_slim.html\n\nｅＭＡＸＩＳ Ｓｌｉｍ 全世界株式（オール・カントリー）\n\n追加型投信／内外／株式／インデックス型\n\nファンドは、ＮＩＳＡの成長投資枠およびつみたて投資枠の対象です。 ※販売会社により取扱いが異なる場合があります。くわしくは、販売会社にご確認ください。\n\n※ご購入に際しては、本書の内容を十分にお読みください。\n\n商品分類\n\n属性区分\n\n単位型・ 追加型\n\n投資対象 地域\n\n投資対象資産 （収益の源泉）\n\n補足分類\n\n投資対象 資産\n\n決算 頻度\n\n投資対象 地域\n\n投資 形態\n\n為替 ヘッジ\n\n対象インデックス\n\n追加型\n\n内外\n\n株式\n\nインデックス 型\n\nその他 資産\n\n年１回\n\nグローバル （日本を含む）\n\nファミリー ファンド\n\nなし\n\nその他（ＭＳＣＩ オール・カント リー・ワールド・インデックス （配当込み、円換算ベース））\n\n※属性区分の「投資対象資産」に記載されている「その他資産」とは、投資信託証券（株式\u3000一般）です。 ※属性区分に記載している「為替ヘッジ」は、対円での為替リスクに対するヘッジの有無を記載しています。 ※商品分類および属性区分の内容については、一般社団法人投資信託協会のホームページ（https://www.toushin.or.jp/）でご覧いただけます。\n\n本書は、金融商品取引法第13条の規定に基づく目論見書です。\n\nファンドに関する投資信託説明書（請求目論見書）を含む詳細な情報は、委託会社の ホームページで閲覧、ダウンロードできます。\n\n委託会社：三菱ＵＦＪアセットマネジメント株式会社 ファンドの運用の指図等を行

In [4]:
# https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)
docs = text_splitter.split_documents(raw_docs)
docs

[Document(page_content='投資信託説明書 （交付目論見書）\n\n使用開始日 2024.１.19\n\n自分だけの投資スタイル、探せる、見つかる。\n\nノーロード・インデックスファンド・シリーズ。\n\nｅＭＡＸＩＳ専用サイト\u3000 https://emaxis.jp/\n\nファンドのiDeCo取扱先は「eMAXIS SlimシリーズiDeCo取扱先」まで https://www.am.mufg.jp/service/ideco/sales_slim.html\n\nｅＭＡＸＩＳ Ｓｌｉｍ 全世界株式（オール・カントリー）\n\n追加型投信／内外／株式／インデックス型\n\nファンドは、ＮＩＳＡの成長投資枠およびつみたて投資枠の対象です。 ※販売会社により取扱いが異なる場合があります。くわしくは、販売会社にご確認ください。\n\n※ご購入に際しては、本書の内容を十分にお読みください。\n\n商品分類\n\n属性区分\n\n単位型・ 追加型\n\n投資対象 地域\n\n投資対象資産 （収益の源泉）\n\n補足分類\n\n投資対象 資産\n\n決算 頻度\n\n投資対象 地域\n\n投資 形態\n\n為替 ヘッジ\n\n対象インデックス\n\n追加型\n\n内外\n\n株式\n\nインデックス 型\n\nその他 資産', metadata={'source': 'doc/sample.pdf'}),
 Document(page_content='為替 ヘッジ\n\n対象インデックス\n\n追加型\n\n内外\n\n株式\n\nインデックス 型\n\nその他 資産\n\n年１回\n\nグローバル （日本を含む）\n\nファミリー ファンド\n\nなし\n\nその他（ＭＳＣＩ オール・カント リー・ワールド・インデックス （配当込み、円換算ベース））\n\n※属性区分の「投資対象資産」に記載されている「その他資産」とは、投資信託証券（株式\u3000一般）です。 ※属性区分に記載している「為替ヘッジ」は、対円での為替リスクに対するヘッジの有無を記載しています。 ※商品分類および属性区分の内容については、一般社団法人投資信託協会のホームページ（https://www.toushin.or.jp/）でご覧いただけます。\n\n本書は、金融商品取

In [5]:
embeddings = OpenAIEmbeddings()

In [6]:
chroma = Chroma.from_documents(documents=docs, embedding=embeddings, persist_directory="vector")

In [7]:
chroma

In [8]:
query = "運用利回り"
result = chroma.similarity_search(query)
result

[Document(page_content='運用管理費用\n\n1兆円以上の部分\n\n0.05753％ 0.0523％ 0.0173％ 0.0175％ 0.0175％\n\n（信託報酬）\n\n※上記各支払先への配分には、別途消費税等相当額がかかります。\n\n（ご参考：上記信託報酬率を用いて計算したファンドの純資産総額ごとの実質信託報酬率の例）\n\nファンドの純資産総額\n\n１兆1,000億円 １兆6,000億円 ２兆1,000億円\n\n実質信託報酬率（税込 年率）\n\n0.05768%\n\n0.05764%\n\n0.05761%\n\n＜各支払先が運用管理費用（信託報酬）の対価として提供する役務の内容＞\n\n支払先\n\n対価として提供する役務の内容\n\n委託会社 ファンドの運用・調査、受託会社への運用指図、基準価額の算出、目論見書等の作成等\n\n販売会社 交付運用報告書等各種書類の送付、顧客口座の管理、購入後の情報提供等\n\n受託会社 ファンドの財産の保管および管理、委託会社からの運用指図の実行等\n\nその他の費用・ 手数料', metadata={'source': 'doc/sample.pdf'}),
 Document(page_content='※確定拠出年金法に定める加入者等の運用の指図に基づいて購入の申込みを行う資産管理機関および国民年金基金連合会等 の場合、所得税および地方税がかかりません。なお、確定拠出年金制度の加入者については、確定拠出年金の積立金の運用に かかる税制が適用されます。\n\n※分配時において、外国税額控除の適用となった場合には、分配時の税金が上記と異なる場合があります。 ※法人の場合は上記とは異なります。 ※税法が改正された場合等には、税率等が変更される場合があります。税金の取扱いの詳細については、税務専門家等に確認さ\n\nれることをお勧めします。\n\n（参考情報）ファンドの総経費率\n\n直近の運用報告書作成対象期間（以下「当期間」といいます。）（2022年４月26日～2023年４月25日）における 当ファンドの総経費率は以下の通りです。\n\n総経費率（①＋②）\n\n①運用管理費用の比率\n\n②その他費用の比率\n\n0.15％\n\n0.11％\n\n0.03％\n\n（比率は年率、